# IMPORTS

In [ ]:
import os
from os import path
import json
import collections

import numpy as np
import pandas as pd
import cv2
from PIL import Image
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from torchvision.utils import make_grid
from torchvision import datasets, transforms, models
from torch.utils import data as torch_data
from tqdm import tqdm
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

* The below statement will help in better error intepretation for CUDA.

In [ ]:
CUDA_LAUNCH_BLOCKING="1"

# Data Loading and Visualization


In [ ]:
base_path = '../input/herbarium-2021-fgvc8'
train_path = os.path.join(base_path, "train/")
train_metadata_path = os.path.join(train_path, "metadata.json")
test_path = os.path.join(base_path, "test/")
test_metadata_path = os.path.join(test_path, "metadata.json")

In [ ]:
with open(train_metadata_path) as json_file:
    metadata = json.load(json_file)
    
metadata.keys()

In [ ]:
print(metadata["annotations"][0])
print(metadata["images"][0])
print(metadata["categories"][0])
print(metadata["licenses"][0])
print(metadata["institutions"][0])

In [ ]:
class Create_Image_Paths():
    def __init__(self,datafile):
        with open(datafile) as json_file:
            self.metadata = json.load(json_file)
    
    def create_dataframe(self):
        ids = []
        categories = []
        paths = []

        for annotation, image in zip(metadata["annotations"], metadata["images"]):
            assert annotation["image_id"] == image["id"]
            ids.append(image["id"])
            categories.append(annotation["category_id"])
            paths.append(image["file_name"])
        
        self.df = pd.DataFrame({"id": ids, "category": categories, "path": paths})
        d_categories = {category["id"]: category["name"] for category in metadata["categories"]}
        d_families = {category["id"]: category["family"] for category in metadata["categories"]}
        d_orders = {category["id"]: category["order"] for category in metadata["categories"]}
        self.df["category_name"] = self.df["category"].map(d_categories)
        self.df["family_name"] = self.df["category"].map(d_families)
        self.df["order_name"] = self.df["category"].map(d_orders)
        

In [ ]:
Image_Paths_Obj = Create_Image_Paths(train_metadata_path)
Image_Paths_Obj.create_dataframe()
Image_Paths_Obj.df.head()

In [ ]:
n_class = len(Image_Paths_Obj.df.groupby('category'))
n_class

In [ ]:
Image_Paths_Obj.df.path[0]

In [ ]:
class Data_Visualization:
    def __init__(self,df):
        self.dfx = df
        
    def visualize_by_id(self, _id=None):
        tmp = self.dfx.sample(6)
        if _id is not None:
            tmp = self.dfx[self.dfx["category"] == _id].sample(6)
            
        self.visualize_train_batch(
            tmp["path"].tolist(), 
            tmp["category_name"].tolist(),
            tmp["family_name"].tolist(),
            tmp["order_name"].tolist())
     
    def visualize_train_batch(self,paths, categories, families, orders):
        plt.figure(figsize=(16, 16))
    
        for ind, info in enumerate(zip(paths, categories, families, orders)):
            path, category, family, order = info
        
            plt.subplot(2, 3, ind + 1)
        
            image = cv2.imread(os.path.join(train_path, path))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            plt.imshow(image)
        
            plt.title(
                f"FAMILY: {family} ORDER: {order}\n{category}", 
                fontsize=10,
            )
            plt.axis("off")
    
        plt.show()

In [ ]:
data_viz = Data_Visualization(Image_Paths_Obj.df)
data_viz.visualize_by_id()

# Data Preparation for Model Training

In [ ]:
class DataRetriever(torch_data.Dataset):
    def __init__(self, paths, categories=None,transforms=None,base_path=train_path):
        self.paths = paths
        self.categories = categories
        self.transforms = transforms
        self.base_path = base_path
          
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        img = Image.open(os.path.join(self.base_path, self.paths[index]))
        #img = cv2.resize(img, (224, 224))
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            img = self.transforms(img)
        
        if self.categories is None:
            return img
        
        y = self.categories[index] 
        return img, y
    
    
def get_transforms():
    return transforms.Compose([
        transforms.RandomRotation(10),      # rotate +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # reverse 50% of images
        transforms.Resize(224),             # resize shortest side to 224 pixels
        transforms.CenterCrop(224),         # crop longest side to 224 pixels at center
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
            

In [ ]:
tr_df = Image_Paths_Obj.df
tmp_path = tr_df["path"].tolist()
tmp_category = tr_df["category"].tolist()
BATCH = 10
train_data_retriever = DataRetriever(
    tmp_path,
    tmp_category,
    transforms=get_transforms(),
)

torch.manual_seed(42)
#train_data_loader = DataLoader(train_data_retriever, batch_size=BATCH, shuffle=True)

In [ ]:
Base_Model1 = models.resnet34(pretrained=True)
for param in Base_Model1.parameters():
    param.requires_grad = False

torch.manual_seed(42)
Base_Model1.fc = nn.Linear(512, n_class, bias=True)

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
DEVICE

# Model Training

The model will be trained in K-Fold cross validation in GPU. The learning rate scheduler that will be used is Cosine Annealing.

In [ ]:
%%time
EPOCHS = 10

criterion = nn.CrossEntropyLoss()

In [ ]:
# Define the K-fold Cross Validator
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_data_retriever)):
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      train_data_retriever, 
                      batch_size=BATCH, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      train_data_retriever,
                      batch_size=BATCH, sampler=test_subsampler)
    
    #configure the model for GPU training
    model = Base_Model1.to(DEVICE)
    epochs = 0
    #set the optimizer
    optimizer = torch.optim.Adam(Base_Model1.parameters(), lr=0.01)
    #set the learning rate scheduler
    step_size = 4*len(trainloader)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, step_size)
        
    
    # Run the training loop for defined number of epochs
    for epoch in range(epochs, EPOCHS):
        # Print epoch
        print(f'Starting epoch {epoch+1}')
        
        # Run the training batches
        for b, (X_train, y_train) in enumerate(trainloader):
            
            b+=1
            X_train = X_train.cuda()
            y_train = y_train.cuda()
            # Apply the model
            y_pred = model.forward(X_train)
            loss = criterion(y_pred, y_train)
            
            
            # Update parameters
            optimizer.zero_grad()
            loss.backward()
            scheduler.step()
            optimizer.step()
            
            # Print interim results
            if b%20 == 0:
                print(f'epoch: {epoch:2}  batch: {b:4}  loss: {loss.item():10.8f}') 
                
            
    # Process is complete.
    print('Training process has finished. Saving trained model.')
    
    # Print about testing
    print('Starting testing')
    
    # Saving the model
    save_path = f'model-fold-{fold}.pth'
    torch.save(model.state_dict(), save_path)
    
    # Evaluation for this fold
    correct, total = 0, 0
    with torch.no_grad():
        
        # Iterate over the test data and generate predictions
        for b, (X_test, y_test) in enumerate(testloader):
            b+=1
            X_test = X_test.cuda()
            y_test = y_test.cuda()
            
            # Apply the model
            y_val = model.forward(X_test)
            # Tally the number of correct predictions
            predicted = torch.max(y_val.data, 1)[1] 
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()
            
        
        # Print accuracy
        print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
        print('--------------------------------')
        results[fold] = 100.0 * (correct / total)
            
            
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
sum = 0.0
for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
print(f'Average: {sum/len(results.items())} %')                   